In [ ]:
import os
import io
from getpass import getpass
from BiConn import BiConn
from Auxs   import hashs
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import ParameterFormat
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.primitives.serialization import load_der_parameters
from cryptography.hazmat.primitives.serialization import load_der_public_key

In [ ]:
curve = ec.SECP256K1

def ECDH(conn):

    private_key_R = ec.generate_private_key(curve, default_backend())
    public_key_R = private_key_R.public_key().public_bytes(                          
                        encoding=serialization.Encoding.PEM,
                        format=serialization.PublicFormat.SubjectPublicKeyInfo)

    private_key_E = ec.generate_private_key(curve, default_backend())
    public_key_E = private_key_E.public_key().public_bytes( 
                        encoding=serialization.Encoding.PEM,
                        format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
    conn.send(public_key_E)

    signature = private_key_E.sign(public_key_R, ec.ECDSA(hashes.SHA256()))

    emitter_pub = serialization.load_pem_public_key(
                    conn.recv(), 
                    backend=default_backend())

    conn.send(public_key_R)
    conn.send(signature)
    
    try:
        receiver = conn.recv()
        sign = conn.recv()
        emitter_pub.verify(sign, receiver, ec.ECDSA(hashes.SHA256()))
        print("Sucess")
    except InvalidSignature:
        print("Something went wrong!")
        
    public_key = serialization.load_pem_public_key(
                    receiver,
                    backend=default_backend())
    shared_key = private_key_R.exchange(ec.ECDH(), public_key)

    # confirmation
    tag = hashs(bytes(shared_key))
    conn.send(tag)
    peer_tag = conn.recv()
    if peer_tag == tag:
        print('all good')
        return tag
    else:
        print('not good')

    private_key_R = None
    public_key_R = None
    receiver = None
    public_key = None
    shared_key = None
    tag = None
    peer_tag = None

In [ ]:
msg_size = 2**10

def Emitter(conn):
    key = ECDH(conn)
    
    inputs = io.BytesIO(bytes('7'*msg_size,'utf-8'))
    
    cipher_key = os.urandom(32)
    nounce = os.urandom(12)
    
    cipher = ChaCha20Poly1305(cipher_key)

    conn.send(cipher_key)
    conn.send(nounce)
    buffer = bytearray(32)

    try:     
        while inputs.readinto(buffer): 
            ciphertext = cipher.encrypt(nounce,bytes(buffer),None)
            conn.send((ciphertext))         
    except Exception as err:
        print("Error: {0}".format(err))

    inputs.close()
    conn.close()

    key = None

In [ ]:
def Receiver(conn):

    key = ECDH(conn)
    
    outputs = io.BytesIO()
    
    cipher_key = conn.recv()
    nounce = conn.recv()
    
    cipher = ChaCha20Poly1305(cipher_key)
    
    try:
        while True:
            try:
                buffer = conn.recv()
                ciphertext = bytes(buffer)
                message = cipher.decrypt(nounce,ciphertext,None)
                outputs.write(message)
                #print(outputs.getvalue())
            except Exception as err:
                print("Error 1: {0}".format(err))
    except Exception as err:
        print("Error 3: {0}".format(err))

    outputs.close()
    conn.close()
    
    key = None

In [ ]:
BiConn(Emitter, Receiver, timeout=30).auto()